In [1]:
!pip install --upgrade google-cloud-storage

     -------------------------------------- 107.0/107.0 kB 1.5 MB/s eta 0:00:00
     ---------------------------------------- 76.9/76.9 kB 4.2 MB/s eta 0:00:00
     -------------------------------------- 167.2/167.2 kB 3.3 MB/s eta 0:00:00
     -------------------------------------- 114.6/114.6 kB 7.0 MB/s eta 0:00:00
     ------------------------------------- 211.7/211.7 kB 12.6 MB/s eta 0:00:00
     -------------------------------------- 525.5/525.5 kB 8.2 MB/s eta 0:00:00
     -------------------------------------- 155.3/155.3 kB 9.1 MB/s eta 0:00:00
     ---------------------------------------- 77.1/77.1 kB 4.2 MB/s eta 0:00:00



[notice] A new release of pip available: 22.2.1 -> 22.2.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
! pip install allennlp

In [ ]:
pip install allennlp-models

In [ ]:
pip install cached-path==1.1.2

In [ ]:
!pip install -U spacy

In [ ]:
from allennlp.data import Instance
from allennlp.data.token_indexers import TokenIndexer
from allennlp.data.tokenizers import Token
from allennlp.nn import util as nn_util
from allennlp.common.checks import ConfigurationError
import allennlp
import torch
import allennlp_models
from allennlp.predictors.predictor import Predictor
import allennlp_models.tagging
from allennlp_models.coref.models import CoreferenceResolver
from allennlp_models.coref.dataset_readers import conll
from allennlp_models.pretrained import load_predictor
import json
from tqdm import tqdm
import pickle as pkl
import spacy
from allennlp.data.tokenizers import spacy_tokenizer 
# from allennlp.predictors.predictor import Predictorpredictor
USE_GPU = torch.cuda.is_available()

## Coreferencing using Allennlp

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
predicted = load_predictor(model_id="coref-spanbert",cuda_device=0)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
lerc is not a registered model.
downloading: 100%|##########| 1.25G/1.25G [00:21<00:00, 63.4MiB/s]


Downloading:   0%|          | 0.00/414 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/634M [00:00<?, ?B/s]

Some weights of BertModel were not initialized from the model checkpoint at SpanBERT/spanbert-large-cased and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.7/dist-packages/spacy/util.py:865: UserWarning: [W095] Model 'en_core_web_sm' (3.3.0) was trained with spaCy v3.3 and may not be 100% compatible with the current version (3.4.0). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [ ]:
train_data = json.load(open("/content/drive/MyDrive/Docred_documents/train_annotated.json",'r'))

In [ ]:
train_data[2].keys()

dict_keys(['vertexSet', 'labels', 'title', 'sents'])

In [ ]:
train_data[0]['sents'][0]

In [ ]:
train_data[0]['vertexSet']

[[{'name': 'Zest Airways, Inc.', 'pos': [0, 4], 'sent_id': 0, 'type': 'ORG'},
  {'name': 'Asian Spirit and Zest Air',
   'pos': [10, 15],
   'sent_id': 0,
   'type': 'ORG'},
  {'name': 'AirAsia Zest', 'pos': [6, 8], 'sent_id': 0, 'type': 'ORG'},
  {'name': 'AirAsia Zest', 'pos': [19, 21], 'sent_id': 6, 'type': 'ORG'}],
 [{'name': 'Ninoy Aquino International Airport',
   'pos': [4, 8],
   'sent_id': 3,
   'type': 'LOC'},
  {'name': 'Ninoy Aquino International Airport',
   'pos': [26, 30],
   'sent_id': 0,
   'type': 'LOC'}],
 [{'name': 'Pasay City', 'pos': [31, 33], 'sent_id': 0, 'type': 'LOC'}],
 [{'name': 'Metro Manila', 'pos': [34, 36], 'sent_id': 0, 'type': 'LOC'}],
 [{'name': 'Philippines', 'pos': [38, 39], 'sent_id': 0, 'type': 'LOC'},
  {'name': 'Philippines', 'pos': [13, 14], 'sent_id': 4, 'type': 'LOC'},
  {'name': 'Republic of the Philippines',
   'pos': [25, 29],
   'sent_id': 5,
   'type': 'LOC'}],
 [{'name': 'Manila', 'pos': [13, 14], 'sent_id': 1, 'type': 'LOC'},
  {'name'

In [ ]:
from collections import OrderedDict
from collections import Counter

In [ ]:
print(k)

[{'pos': [8, 10], 'type': 'TIME', 'sent_id': 7, 'name': 'January 2016'}]


In [ ]:
from collections import OrderedDict

In [ ]:
def get_cluster(data,clustr_dict):
  sent_ids=[]
  for i,j in enumerate(data):
    sent_ids.append(j['sent_id'])
  sent_ids_cnt=Counter(sent_ids)
  values=list(sent_ids_cnt.values())
  max_val=max(values)
  if max_val>1:
    for i,j in enumerate(data):
      if j['sent_id']==max_val:
        cls_dict={}
        for k,l in enumerate(j):
          

17

In [ ]:
k

[{'name': 'January 2016', 'pos': [8, 10], 'sent_id': 7, 'type': 'TIME'}]

In [ ]:
fl=open("docred_stat_.csv","w")
import csv
fl_csv=csv.writer(fl)
column=["Title","Actual Entity","Coref Output"]
fl_csv.writerow(column)

34

In [ ]:
for i in tqdm(train_data):
  row=[]
  sents=[]
  names=[]
  pred_names=[]
  for l in i['sents']:
    tmp_sent=" ".join(l)
    sents.append(tmp_sent)
  for j in i['vertexSet']:
    temp=[]
    for k in j:
      temp.append(k['name'])
    if len(temp)>1:
      names.append(temp)
  sntce=" ".join(sents)
  output = predicted.predict(document=sntce)
  cluster_map = {}
  clusters = output['clusters']
  doc = output['document']
  for ind,cl in enumerate(clusters):
    lst_ = []
    for k_ in cl:
      # print(k)
      sent_list = doc[k_[0]:k_[-1]+1]
      # print(sent_list)
      lst_.append(" ".join(sent_list))
    pred_names.append(lst_)
  row.append(i['title'])
  #row.append(sntce)
  row.append(names)
  row.append(pred_names)
  fl_csv.writerow(row)

  5%|▍         | 140/3053 [00:25<08:52,  5.47it/s]/usr/local/lib/python3.7/dist-packages/allennlp/modules/token_embedders/pretrained_transformer_embedder.py:385: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  num_effective_segments = (seq_lengths + self._max_length - 1) // self._max_length
100%|██████████| 3053/3053 [09:13<00:00,  5.52it/s]


In [ ]:
fl.close()

In [ ]:
train_data

In [ ]:
'document_title':['zest':[j=hg],'ninoy':[hhjh]]